# Holt-Winters

In [1]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.express as px
import plotly.graph_objects as go

import itertools
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

In [3]:
dados = (pd.read_csv('dataset/dataset_total.csv').set_index('Data').Consumo/1000)[:-36]

In [11]:
def HW(series, params):
    t,d,s,c = params
    model = ExponentialSmoothing(series, 
                                trend=t, damped_trend=d, seasonal=s, seasonal_periods=12, use_boxcox=c, 
                                initialization_method='estimated', missing='drop').fit()
    return (model)

In [12]:
#Hiperparâmetros do Modelo
t = ['add'] 
s = ['add', 'mul'] 
d = [True, False]
c = [False, True]

s = [i for i in list(itertools.product(*[t,d,s,c])) if not(i[0]==None and i[1]==True)]

In [6]:
#Preparação dos Dados
def split(dados, test_size, jump, n_splits):
    splits = []
    for i in range(n_splits):
        if i==0:
            train = dados[:][:-test_size]
            test = dados[:][-test_size:]
        else:
            train = dados[:-i*jump][:-test_size]
            test = dados[:-i*jump][-test_size:]
        splits.append((train,test))
    return splits


In [7]:
#Validação Cruzada com Janela Deslizante
df = {}
for param in s:
    df[param] = []
    for train, test in split(dados,1,1,36):
        model = HW(train, param)
        y_pred = model.forecast(len(test))
        df[param].append((mean_squared_error(test,y_pred, squared=False),mean_absolute_percentage_error(test,y_pred)*100))
    df[param] = np.mean(df[param], axis=0)

result = pd.DataFrame()
result['Order'] = df.keys()
result['MAPE'] = [i[1] for i in df.values()]
result['RMSE'] = [i[0] for i in df.values()]

result.sort_values('MAPE')

,Order,MAPE,RMSE
5,"(add, False, add, True)",1.670882,234.459900
1,"(add, True, add, True)",1.682427,235.981101
3,"(add, True, mul, True)",1.743290,244.025380
2,"(add, True, mul, False)",1.744502,244.419071
7,"(add, False, mul, True)",1.746642,244.388626
4,"(add, False, add, False)",1.755358,246.038860
0,"(add, True, add, False)",1.755785,245.962568
6,"(add, False, mul, False)",1.765634,247.351629


In [17]:
#Previsão do conjunto de validação
model = HW(dados[:-36].values, (result.sort_values('MAPE'))['Order'].iloc[0])
y_pred = model.forecast(36)
print('MAPE',mean_absolute_percentage_error(dados[-36:].values,y_pred)*100,'\nRMSE',mean_squared_error(dados[-36:].values,y_pred, squared=False))

fig = px.line(dados[-36:], labels={'value': 'Consumo (GWh)', 'variable': 'Dados'}, markers=True)
fig.add_trace(go.Scatter(mode='lines', x= dados[-36:].index, y=y_pred, name='Previsto'))

MAPE 1.50418766765292 
RMSE 261.5041347721644
